In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import classification_report, accuracy_score, make_scorer


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline


In [2]:
column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")
df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [3]:
df.loc[df.duygu=="olumlu","duygu"]=1
df.loc[df.duygu=="olumsuz","duygu"]=0
df[:4]

,tweets,duygu
0,turkcell heryerde çekiyor kesin bilgi,1
1,turkcell olmak ayrıcalıktir çünkü kuzenlerin v...,1
2,allahtan turkcell'liyim amin,1
3,avea kaşar yaşasın turkcell,1


,tweets,duygu
0,turkcell heryerde çekiyor kesin bilgi,1
1,turkcell olmak ayrıcalıktir çünkü kuzenlerin v...,1
2,allahtan turkcell'liyim amin,1
3,avea kaşar yaşasın turkcell,1


In [4]:
df=df.sample(frac=1).reset_index(drop=True)
df

,tweets,duygu
0,denizin ortasında akıllı telefonuyla kaybolan ...,0
1,bugün mutluydum ama bu turkcell beni sinir ett...,0
2,ben neden sabahtan beri 3g kullanamiyorum acab...,0
3,bu turkcell in allah cezasını versin yav???? n...,0
4,﻿#şuanhissedereksöylüyorumki turkcell superonl...,0
...,...,...
9121,ucrersiz olarak hem tablet hem de turkcell tv ...,1
9122,turkcell köyde telefonu aldım elime define ara...,0
9123,şu vodafone u ilk aldığım zamanla şimdi arasın...,1
9124,turkcell bugun ananeme yeni telefon satmis.. b...,1


,tweets,duygu
0,denizin ortasında akıllı telefonuyla kaybolan ...,0
1,bugün mutluydum ama bu turkcell beni sinir ett...,0
2,ben neden sabahtan beri 3g kullanamiyorum acab...,0
3,bu turkcell in allah cezasını versin yav???? n...,0
4,﻿#şuanhissedereksöylüyorumki turkcell superonl...,0
...,...,...
9121,ucrersiz olarak hem tablet hem de turkcell tv ...,1
9122,turkcell köyde telefonu aldım elime define ara...,0
9123,şu vodafone u ilk aldığım zamanla şimdi arasın...,1
9124,turkcell bugun ananeme yeni telefon satmis.. b...,1


In [5]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [6]:
X_t=df['tweets'].tolist()

In [7]:
# Importing library
from keras.preprocessing.text import Tokenizer
# Creating a tokenizer
tokenizer = Tokenizer(lower=True)
# Building word indices
tokenizer.fit_on_texts(X_t)
# Tokenizing sentences
sentences = tokenizer.texts_to_sequences(X_t)
# Creating a reverse dictionary
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)
# Creating texts 
X_t = list(map(sequence_to_text, sentences))


In [8]:
#!pip install glove-python-binary

In [9]:
#importing the glove library
from glove import Corpus, Glove
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(X_t, window=5)
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=32, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=50, no_threads=8, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 50 training epochs with 8 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3Performing 50 training epochs with 8 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12

Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19

Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40

Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49

Epoch 49


In [10]:
wordembeddings=glove.load('glove.model')

In [11]:
def glove (X_train,y_train,X_test,y_test):
    #Create a tokenizer, configured to only take into account the 20 most common words çok küçük olursa kelimeleri 
    #kaybederiz underfit yaparız
    #tokenizer = Tokenizer(lower=True) #en yaygın kaç kelimeyi dikkate alacağı. Belirtilecek en iyi kelime sayısı #1000 yapan da var
    tokenizer.fit_on_texts(X_train) #keras tokenizer ile metni dictionary haline getiriyor.
    sequences_X_train = tokenizer.texts_to_sequences(X_train) #kelimelerin dictionarydeki karşılığı 
    #[[2, 1, 3], [2, 1], [4, 1], [5, 6]] şekline getiriliyor. 2-machine 1- learning 3-Knowledge 
    word_index = tokenizer.word_index #dictionarydeki kelimelerin sayısal karşılığı 'unk': 1, 'ürün': 2,
    max_length = 0
    for review_number in range(len(sequences_X_train)): #len(sequences_X_train) ile kaç tane [[2,3,4],[2,6]] var bulunuyor burda 2
        numberofwords=len(sequences_X_train[review_number]) #[2,3,4] içinde kaç tane şey var 3 burda
        if (numberofwords) > (max_length):
            max_length = numberofwords #tüm kelimelere bakıp en uzun olanı buluyor

    X_train = pad_sequences(sequences_X_train, maxlen=max_length) #ikili boyutlu matrise çevirip her cümelnin uzunluğunu eşit yapıyor.
    #En uzun cümle uzunluğuna tamamlanıyor.[[2 1 3] [0 2 1]] alt alta gelecek şekilde en uzun 6 ise 6x6 matris oluyor
    y_train = np.asarray(y_train) #tek boyutlu bir matris oluyor [1 1 0 ... 0 1 0] gibi

    sequences_X_test = tokenizer.texts_to_sequences(X_test) #train için yapılan gibi dictionary alınıyor
    X_test = pad_sequences(sequences_X_test, maxlen=max_length) #en uzun olana göre pad sequence yapılıyor
    y_test = np.asarray(y_test)


    unique_words = len(word_index) #word_index ile unique olan kelimeler alınıyor 0 dan başladığı için bir arttırılıyor
    total_words = unique_words + 1
    skipped_words = 0
    embedding_dim = 32 #embedding dim vector size ile aynı 
    embedding_vector=0
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items(): #kelime ve kelimenin dictionarydeki karşılığı alınıyor
        try:
            embedding_vector = wordembeddings.word_vectors[wordembeddings.dictionary[word]] #kelimenin word2vec karşılığı vektör olarak
        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector #dictionarydeki indexine word2vec teki sayısal hali yazılır
            
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)
    
    return embedding_layer,X_train,y_train,X_test,y_test

In [12]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= glove (X_train,y_train,X_test,y_test)
    
    pat = 5
    early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=1)
    model = Sequential() #rnn
    model.add(embedding_layer)
    model.add(SimpleRNN(128,activation='relu',return_sequences= True))
    model.add(SimpleRNN(256,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    model.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 9s 36ms/step - loss: 0.6315 - acc: 0.6378 - precision: 0.6494 - recall: 0.5967
Epoch 2/50
Epoch 2/50
257/257 [==============================] - 10s 37ms/step - loss: 0.5940 - acc: 0.6742 - precision: 0.6958 - recall: 0.6174
Epoch 3/50
Epoch 3/50
257/257 [==============================] - 10s 39ms/step - loss: 0.5842 - acc: 0.6850 - precision: 0.7077 - recall: 0.6289
Epoch 4/50
Epoch 4/50
257/257 [==============================] - 9s 35ms/step - loss: 0.5649 - acc: 0.7034 - precision: 0.7235 - recall: 0.6572
Epoch 5/50
Epoch 5/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5594 - acc: 0.7080 - precision: 0.7274 - recall: 0.6642
Epoch 6/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5433 - 

Epoch 26/50
257/257 [==============================] - 9s 33ms/step - loss: 0.2029 - acc: 0.9137 - precision: 0.9159 - recall: 0.9108
Epoch 27/50
257/257 [==============================] - 9s 34ms/step - loss: 0.2164 - acc: 0.9084 - precision: 0.9146 - recall: 0.9008
Epoch 28/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1564 - acc: 0.9344 - precision: 0.9378 - recall: 0.9303
Epoch 29/50
Epoch 29/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1815 - acc: 0.9257 - precision: 0.9279 - recall: 0.9229
Epoch 30/50
257/257 [==============================] - 9s 33ms/step - loss: 0.1382 - acc: 0.9418 - precision: 0.9459 - recall: 0.9371
Epoch 31/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1570 - acc: 0.9351 - precision: 0.9401 - recall: 0.9293
Epoch 32/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1649 - acc: 0.9321 - precision: 0.9309 - recall: 0.9332
Epoch 33/50
257/257 [=============================

Epoch 2/50
Epoch 2/50
257/257 [==============================] - 9s 33ms/step - loss: 0.5943 - acc: 0.6732 - precision_1: 0.7012 - recall_1: 0.5979
Epoch 3/50
Epoch 3/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5885 - acc: 0.6745 - precision_1: 0.6983 - recall_1: 0.6089
Epoch 4/50
257/257 [==============================] - 8s 33ms/step - loss: 0.5701 - acc: 0.6940 - precision_1: 0.7158 - recall_1: 0.6385
Epoch 5/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5538 - acc: 0.7105 - precision_1: 0.7309 - recall_1: 0.6615
Epoch 6/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5419 - acc: 0.7187 - precision_1: 0.7382 - recall_1: 0.6735
Epoch 7/50
Epoch 7/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5282 - acc: 0.7252 - precision_1: 0.7445 - recall_1: 0.6816
Epoch 8/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5187 - acc: 0.7308 - precision_1: 0.7541 - recall_1: 0.6809
Epoch 9/

257/257 [==============================] - 9s 34ms/step - loss: 0.1687 - acc: 0.9283 - precision_1: 0.9287 - recall_1: 0.9271
Epoch 26/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1596 - acc: 0.9334 - precision_1: 0.9340 - recall_1: 0.9320
Epoch 27/50
Epoch 27/50
257/257 [==============================] - 9s 33ms/step - loss: 0.1645 - acc: 0.9321 - precision_1: 0.9341 - recall_1: 0.9290
Epoch 28/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1454 - acc: 0.9392 - precision_1: 0.9432 - recall_1: 0.9342
Epoch 29/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1487 - acc: 0.9368 - precision_1: 0.9381 - recall_1: 0.9347
Epoch 30/50
Epoch 30/50
257/257 [==============================] - 9s 35ms/step - loss: 0.1252 - acc: 0.9492 - precision_1: 0.9508 - recall_1: 0.9469
Epoch 31/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1527 - acc: 0.9351 - precision_1: 0.9403 - recall_1: 0.9285
Epoch 32/50
Epoch 32/5

257/257 [==============================] - 9s 34ms/step - loss: 0.5797 - acc: 0.6883 - precision_2: 0.7101 - recall_2: 0.6289
Epoch 4/50
257/257 [==============================] - 9s 33ms/step - loss: 0.5638 - acc: 0.7018 - precision_2: 0.7235 - recall_2: 0.6463
Epoch 5/50
Epoch 5/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5516 - acc: 0.7146 - precision_2: 0.7377 - recall_2: 0.6596
Epoch 6/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5364 - acc: 0.7229 - precision_2: 0.7412 - recall_2: 0.6787
Epoch 7/50
Epoch 7/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5218 - acc: 0.7355 - precision_2: 0.7539 - recall_2: 0.6936
Epoch 8/50
Epoch 8/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5139 - acc: 0.7407 - precision_2: 0.7640 - recall_2: 0.6909
Epoch 9/50
Epoch 9/50
257/257 [==============================] - 9s 33ms/step - loss: 0.4942 - acc: 0.7533 - precision_2: 0.7727 - recall_2: 0.7128
Epoch 10

...

Epoch 43/50
Epoch 43/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1278 - acc: 0.9511 - precision_7: 0.9505 - recall_7: 0.9519
Epoch 44/50
Epoch 44/50
257/257 [==============================] - 9s 35ms/step - loss: 0.1185 - acc: 0.9508 - precision_7: 0.9512 - recall_7: 0.9507
Epoch 45/50
257/257 [==============================] - 9s 33ms/step - loss: 0.0943 - acc: 0.9632 - precision_7: 0.9645 - recall_7: 0.9621
Epoch 46/50
257/257 [==============================] - 9s 35ms/step - loss: 0.1288 - acc: 0.9485 - precision_7: 0.9518 - recall_7: 0.9451
Epoch 47/50
Epoch 47/50
257/257 [==============================] - 9s 35ms/step - loss: 0.0849 - acc: 0.9686 - precision_7: 0.9714 - recall_7: 0.9658
Epoch 48/50
257/257 [==============================] - 9s 34ms/step - loss: 0.0803 - acc: 0.9726 - precision_7: 0.9751 - recall_7: 0.9701
Epoch 49/50
257/257 [==============================] - 9s 34ms/step - loss: 0.0657 - acc: 0.9777 - precision_7: 0.9812 - recall_7: 0.974

257/257 [==============================] - 9s 34ms/step - loss: 0.3635 - acc: 0.8299 - precision_8: 0.8322 - recall_8: 0.8279
Epoch 16/50
Epoch 16/50
257/257 [==============================] - 9s 34ms/step - loss: 0.3316 - acc: 0.8496 - precision_8: 0.8507 - recall_8: 0.8493
Epoch 17/50
257/257 [==============================] - 9s 35ms/step - loss: 0.3155 - acc: 0.8568 - precision_8: 0.8610 - recall_8: 0.8522
Epoch 18/50
Epoch 18/50
257/257 [==============================] - 9s 34ms/step - loss: 0.2984 - acc: 0.8645 - precision_8: 0.8690 - recall_8: 0.8595
Epoch 19/50
257/257 [==============================] - 9s 34ms/step - loss: 0.2690 - acc: 0.8796 - precision_8: 0.8840 - recall_8: 0.8748
Epoch 20/50
Epoch 20/50
257/257 [==============================] - 9s 34ms/step - loss: 0.2496 - acc: 0.8897 - precision_8: 0.8916 - recall_8: 0.8881
Epoch 21/50
257/257 [==============================] - 9s 35ms/step - loss: 0.2274 - acc: 0.8992 - precision_8: 0.9009 - recall_8: 0.8978
Epoch 22/5

257/257 [==============================] - 9s 34ms/step - loss: 0.0831 - acc: 0.9691 - precision_8: 0.9717 - recall_8: 0.9665
Epoch 39/50
257/257 [==============================] - 9s 35ms/step - loss: 0.1216 - acc: 0.9475 - precision_8: 0.9504 - recall_8: 0.9447
Epoch 40/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1518 - acc: 0.9369 - precision_8: 0.9388 - recall_8: 0.9352
Epoch 41/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1088 - acc: 0.9559 - precision_8: 0.9581 - recall_8: 0.9539
Epoch 42/50
257/257 [==============================] - 9s 34ms/step - loss: 0.0830 - acc: 0.9652 - precision_8: 0.9682 - recall_8: 0.9621
Epoch 43/50
Epoch 43/50
257/257 [==============================] - 9s 35ms/step - loss: 0.1058 - acc: 0.9558 - precision_8: 0.9594 - recall_8: 0.9522
Epoch 44/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1306 - acc: 0.9495 - precision_8: 0.9513 - recall_8: 0.9478
Epoch 45/50
257/257 [=============

257/257 [==============================] - 9s 35ms/step - loss: 0.4911 - acc: 0.7531 - precision_9: 0.7640 - recall_9: 0.7305
Epoch 11/50
Epoch 11/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4755 - acc: 0.7588 - precision_9: 0.7712 - recall_9: 0.7341
Epoch 12/50
Epoch 12/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4555 - acc: 0.7753 - precision_9: 0.7880 - recall_9: 0.7515
Epoch 13/50
257/257 [==============================] - 9s 35ms/step - loss: 0.4392 - acc: 0.7816 - precision_9: 0.7953 - recall_9: 0.7568
Epoch 14/50
Epoch 14/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4250 - acc: 0.7918 - precision_9: 0.8086 - recall_9: 0.7632
Epoch 15/50
257/257 [==============================] - 9s 34ms/step - loss: 0.3986 - acc: 0.8101 - precision_9: 0.8231 - recall_9: 0.7886
Epoch 16/50
257/257 [==============================] - 9s 34ms/step - loss: 0.3734 - acc: 0.8236 - precision_9: 0.8382 - recall_9: 0.8008
Epoch 17/5

257/257 [==============================] - 9s 34ms/step - loss: 0.1471 - acc: 0.9389 - precision_9: 0.9351 - recall_9: 0.9429
Epoch 34/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1493 - acc: 0.9388 - precision_9: 0.9426 - recall_9: 0.9341
Epoch 35/50
257/257 [==============================] - 9s 35ms/step - loss: 0.1206 - acc: 0.9534 - precision_9: 0.9556 - recall_9: 0.9507
Epoch 36/50
257/257 [==============================] - 8s 33ms/step - loss: 0.1311 - acc: 0.9455 - precision_9: 0.9460 - recall_9: 0.9446
Epoch 37/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1145 - acc: 0.9530 - precision_9: 0.9562 - recall_9: 0.9492
Epoch 38/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1313 - acc: 0.9477 - precision_9: 0.9495 - recall_9: 0.9453
Epoch 39/50
257/257 [==============================] - 9s 34ms/step - loss: 0.1142 - acc: 0.9539 - precision_9: 0.9545 - recall_9: 0.9529
Epoch 40/50
Epoch 40/50
257/257 [=============

In [13]:
def Average(lst):
    return sum(lst) / len(lst)

In [14]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.673236test Accuracy: 0.673236
test precision: 0.671892
test recall: 0.679257
test f1_score: 0.674660

test precision: 0.671892
test recall: 0.679257
test f1_score: 0.674660


In [15]:
#cnn

In [16]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= glove (X_train,y_train,X_test,y_test)
    
    modelCNN = Sequential() #cnn
    early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
    modelCNN.add(embedding_layer)
    modelCNN.add(Conv1D(filters=256, kernel_size=5, activation='relu')) #kernal size 5 yan yana beş kelimeye bakması
    modelCNN.add(MaxPooling1D(pool_size=2)) #tek satırlık 1d olduğu için
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(360, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(300, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(260, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(150, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(120, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(80, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Flatten()) #düzgünleştirmek için
    modelCNN.add(Dense(1, activation='sigmoid'))
    modelCNN.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNN.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelCNN.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 8s 32ms/step - loss: 0.6681 - acc: 0.5925 - precision_10: 0.5978 - recall_10: 0.5765
Epoch 2/50
257/257 [==============================] - 9s 33ms/step - loss: 0.6376 - acc: 0.6318 - precision_10: 0.6712 - recall_10: 0.5232
Epoch 3/50
Epoch 3/50
257/257 [==============================] - 8s 33ms/step - loss: 0.6224 - acc: 0.6471 - precision_10: 0.6846 - recall_10: 0.5515
Epoch 4/50
257/257 [==============================] - 8s 32ms/step - loss: 0.6123 - acc: 0.6577 - precision_10: 0.6960 - recall_10: 0.5656
Epoch 5/50
257/257 [==============================] - 8s 32ms/step - loss: 0.5996 - acc: 0.6736 - precision_10: 0.7025 - recall_10: 0.6073
Epoch 6/50
257/257 [==============================] - 9s 33ms/step - loss: 0.5980 - acc: 

257/257 [==============================] - 9s 33ms/step - loss: 0.5200 - acc: 0.7325 - precision_10: 0.7477 - recall_10: 0.7055
Epoch 21/50
257/257 [==============================] - 9s 35ms/step - loss: 0.5165 - acc: 0.7304 - precision_10: 0.7412 - recall_10: 0.7118
Epoch 22/50
257/257 [==============================] - 9s 35ms/step - loss: 0.5081 - acc: 0.7362 - precision_10: 0.7504 - recall_10: 0.7113
Epoch 23/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5056 - acc: 0.7416 - precision_10: 0.7565 - recall_10: 0.7161
Epoch 24/50
257/257 [==============================] - 8s 32ms/step - loss: 0.5029 - acc: 0.7403 - precision_10: 0.7488 - recall_10: 0.7268
Epoch 25/50
257/257 [==============================] - 8s 33ms/step - loss: 0.4946 - acc: 0.7488 - precision_10: 0.7609 - recall_10: 0.7290
Epoch 26/50
257/257 [==============================] - 8s 32ms/step - loss: 0.4898 - acc: 0.7531 - precision_10: 0.7619 - recall_10: 0.7394
Epoch 27/50
257/257 [===========

257/257 [==============================] - 8s 32ms/step - loss: 0.4187 - acc: 0.7963 - precision_10: 0.7940 - recall_10: 0.8027
Epoch 42/50
257/257 [==============================] - 8s 33ms/step - loss: 0.4214 - acc: 0.7901 - precision_10: 0.7873 - recall_10: 0.7976
Epoch 43/50
257/257 [==============================] - 8s 31ms/step - loss: 0.4064 - acc: 0.8004 - precision_10: 0.7995 - recall_10: 0.8044
Epoch 44/50
257/257 [==============================] - 8s 33ms/step - loss: 0.4072 - acc: 0.7987 - precision_10: 0.8025 - recall_10: 0.7949
Epoch 45/50
Epoch 45/50
257/257 [==============================] - 8s 32ms/step - loss: 0.3947 - acc: 0.8140 - precision_10: 0.8134 - recall_10: 0.8170
Epoch 46/50
257/257 [==============================] - 8s 32ms/step - loss: 0.4021 - acc: 0.8043 - precision_10: 0.8065 - recall_10: 0.8032
Epoch 47/50
Epoch 47/50
257/257 [==============================] - 8s 31ms/step - loss: 0.3995 - acc: 0.8021 - precision_10: 0.7966 - recall_10: 0.8138
Epoch 48

257/257 [==============================] - 9s 34ms/step - loss: 0.5670 - acc: 0.6972 - precision_11: 0.7236 - recall_11: 0.6389
Epoch 12/50
Epoch 12/50
257/257 [==============================] - 8s 31ms/step - loss: 0.5626 - acc: 0.6963 - precision_11: 0.7179 - recall_11: 0.6477
Epoch 13/50
Epoch 13/50
257/257 [==============================] - 9s 33ms/step - loss: 0.5616 - acc: 0.6957 - precision_11: 0.7262 - recall_11: 0.6292
Epoch 14/50
257/257 [==============================] - 8s 32ms/step - loss: 0.5588 - acc: 0.7038 - precision_11: 0.7283 - recall_11: 0.6509
Epoch 15/50
Epoch 15/50
257/257 [==============================] - 8s 32ms/step - loss: 0.5503 - acc: 0.7125 - precision_11: 0.7329 - recall_11: 0.6696
Epoch 16/50
257/257 [==============================] - 8s 32ms/step - loss: 0.5492 - acc: 0.7067 - precision_11: 0.7231 - recall_11: 0.6708
Epoch 17/50
257/257 [==============================] - 8s 33ms/step - loss: 0.5375 - acc: 0.7208 - precision_11: 0.7412 - recall_11: 0.6

257/257 [==============================] - 8s 32ms/step - loss: 0.4669 - acc: 0.7620 - precision_11: 0.7712 - recall_11: 0.7455
Epoch 32/50
Epoch 32/50
257/257 [==============================] - 8s 32ms/step - loss: 0.4632 - acc: 0.7654 - precision_11: 0.7729 - recall_11: 0.7521
Epoch 33/50Epoch 33/50
257/257 [==============================] - 8s 32ms/step - loss: 0.4571 - acc: 0.7666 - precision_11: 0.7734 - recall_11: 0.7547
Epoch 34/50
257/257 [==============================] - 8s 32ms/step - loss: 0.4513 - acc: 0.7710 - precision_11: 0.7825 - recall_11: 0.7511
Epoch 35/50
257/257 [==============================] - 8s 32ms/step - loss: 0.4442 - acc: 0.7755 - precision_11: 0.7845 - recall_11: 0.7601
Epoch 36/50
Epoch 36/50
257/257 [==============================] - 8s 32ms/step - loss: 0.4430 - acc: 0.7773 - precision_11: 0.7818 - recall_11: 0.7698
Epoch 37/50
257/257 [==============================] - 8s 32ms/step - loss: 0.4341 - acc: 0.7855 - precision_11: 0.7883 - recall_11: 0.78

...

257/257 [==============================] - 9s 34ms/step - loss: 0.6262 - acc: 0.6353 - precision_17: 0.6798 - recall_17: 0.5140
Epoch 4/50
257/257 [==============================] - 9s 35ms/step - loss: 0.6133 - acc: 0.6539 - precision_17: 0.7014 - recall_17: 0.5383
Epoch 5/50
257/257 [==============================] - 9s 36ms/step - loss: 0.6008 - acc: 0.6654 - precision_17: 0.7037 - recall_17: 0.5738
Epoch 6/50
257/257 [==============================] - 13s 51ms/step - loss: 0.5938 - acc: 0.6732 - precision_17: 0.7072 - recall_17: 0.5934
Epoch 7/50
Epoch 7/50
257/257 [==============================] - 9s 37ms/step - loss: 0.5901 - acc: 0.6780 - precision_17: 0.7140 - recall_17: 0.5959
Epoch 8/50
Epoch 8/50
257/257 [==============================] - 9s 35ms/step - loss: 0.5826 - acc: 0.6836 - precision_17: 0.7181 - recall_17: 0.6066
Epoch 9/50
257/257 [==============================] - 9s 33ms/step - loss: 0.5770 - acc: 0.6878 - precision_17: 0.7138 - recall_17: 0.6292
Epoch 10/50
257

257/257 [==============================] - 9s 33ms/step - loss: 0.4823 - acc: 0.7583 - precision_17: 0.7681 - recall_17: 0.7414
Epoch 27/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4758 - acc: 0.7622 - precision_17: 0.7824 - recall_17: 0.7278
Epoch 28/50
Epoch 28/50
257/257 [==============================] - 9s 35ms/step - loss: 0.4759 - acc: 0.7570 - precision_17: 0.7664 - recall_17: 0.7407
Epoch 29/50
Epoch 29/50
257/257 [==============================] - 9s 35ms/step - loss: 0.4762 - acc: 0.7580 - precision_17: 0.7715 - recall_17: 0.7344
Epoch 30/50
Epoch 30/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4775 - acc: 0.7552 - precision_17: 0.7669 - recall_17: 0.7346
Epoch 31/50
Epoch 31/50
257/257 [==============================] - 9s 35ms/step - loss: 0.4640 - acc: 0.7642 - precision_17: 0.7720 - recall_17: 0.7512
Epoch 32/50
Epoch 32/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4548 - acc: 0.7754 - precision_17:

29/29 [==============================] - 0s 8ms/step - loss: 0.7240 - acc: 0.6612 - precision_17: 0.6925 - recall_17: 0.5580
------------------------------------------------------------------------
Training for fold 9 ...
Epoch 1/50
------------------------------------------------------------------------
Training for fold 9 ...
Epoch 1/50
257/257 [==============================] - 9s 36ms/step - loss: 0.6682 - acc: 0.5849 - precision_18: 0.5852 - recall_18: 0.5642
Epoch 2/50
257/257 [==============================] - 10s 37ms/step - loss: 0.6417 - acc: 0.6219 - precision_18: 0.6563 - recall_18: 0.5012
Epoch 3/50
257/257 [==============================] - 10s 39ms/step - loss: 0.6179 - acc: 0.6510 - precision_18: 0.6840 - recall_18: 0.5527
Epoch 4/50
257/257 [==============================] - 10s 39ms/step - loss: 0.6056 - acc: 0.6624 - precision_18: 0.6900 - recall_18: 0.5816
Epoch 5/50
Epoch 5/50
257/257 [==============================] - 10s 40ms/step - loss: 0.6005 - acc: 0.6667 - p

257/257 [==============================] - 15s 60ms/step - loss: 0.5156 - acc: 0.7359 - precision_18: 0.7443 - recall_18: 0.7135
Epoch 22/50
257/257 [==============================] - 16s 63ms/step - loss: 0.5150 - acc: 0.7344 - precision_18: 0.7435 - recall_18: 0.7103
Epoch 23/50
Epoch 23/50
257/257 [==============================] - 21s 83ms/step - loss: 0.5045 - acc: 0.7404 - precision_18: 0.7506 - recall_18: 0.7150
Epoch 24/50
Epoch 24/50
257/257 [==============================] - 26s 102ms/step - loss: 0.4952 - acc: 0.7442 - precision_18: 0.7525 - recall_18: 0.7228
Epoch 25/50
Epoch 25/50
257/257 [==============================] - 28s 107ms/step - loss: 0.5006 - acc: 0.7492 - precision_18: 0.7594 - recall_18: 0.7248
Epoch 26/50
257/257 [==============================] - 28s 107ms/step - loss: 0.5006 - acc: 0.7492 - precision_18: 0.7594 - recall_18: 0.7248
Epoch 26/50
257/257 [==============================] - 27s 106ms/step - loss: 0.4873 - acc: 0.7507 - precision_18: 0.7616 - rec

257/257 [==============================] - 21s 82ms/step - loss: 0.5216 - acc: 0.7236 - precision_19: 0.7435 - recall_19: 0.68350s - loss: 0.5205 - acc: 0.7241 - precision_19: 0.7439 - recal
Epoch 20/50
257/257 [==============================] - 25s 99ms/step - loss: 0.5194 - acc: 0.7274 - precision_19: 0.7533 - recall_19: 0.6769
Epoch 21/50
257/257 [==============================] - 26s 103ms/step - loss: 0.5166 - acc: 0.7243 - precision_19: 0.7516 - recall_19: 0.6706s - loss: 0.5199 - acc: 0.7251 - precision_19: 0.7528 - recall_19 - ETA: 3s -
Epoch 22/50
257/257 [==============================] - 26s 103ms/step - loss: 0.5101 - acc: 0.7424 - precision_19: 0.7663 - recall_19: 0.6981
Epoch 23/50
257/257 [==============================] - 26s 100ms/step - loss: 0.4998 - acc: 0.7412 - precision_19: 0.7637 - recall_19: 0.6990
Epoch 24/50
257/257 [==============================] - 26s 101ms/step - loss: 0.4991 - acc: 0.7414 - precision_19: 0.7613 - recall_19: 0.7039
Epoch 25/50
257/257 [==

In [17]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.670063
test precision: 0.699033
test recall: 0.602209
test f1_score: 0.645075


In [18]:
#lstm

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
pat = 5
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=1)

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= glove (X_train,y_train,X_test,y_test)
    

    modelLSTM = Sequential()
    modelLSTM.add(embedding_layer)
    modelLSTM.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    modelLSTM.add(Dense(1, activation='sigmoid'))
    modelLSTM.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelLSTM.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelLSTM.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 64s 249ms/step - loss: 0.6364 - acc: 0.6314 - precision_20: 0.6392 - recall_20: 0.6070
Epoch 2/50
257/257 [==============================] - 64s 247ms/step - loss: 0.6085 - acc: 0.6593 - precision_20: 0.6707 - recall_20: 0.6287
Epoch 3/50
257/257 [==============================] - 65s 252ms/step - loss: 0.6036 - acc: 0.6629 - precision_20: 0.6711 - recall_20: 0.6416s - loss: 0.6051 - acc: 0.6604 - precision_20: 0.6697 - rec
Epoch 4/50
257/257 [==============================] - 65s 253ms/step - loss: 0.5903 - acc: 0.6765 - precision_20: 0.6894 - recall_20: 0.6447
Epoch 5/50
257/257 [==============================] - 58s 226ms/step - loss: 0.5826 - acc: 0.6787 - precision_20: 0.6900 - recall_20: 0.6513
Epoch 6/50
257/257 [==============================] - 64s 251ms/step - loss: 0.5821 - acc: 0.6826 - precision_20: 0.6956 - recall_20: 0.651

257/257 [==============================] - 55s 213ms/step - loss: 0.6344 - acc: 0.6303 - precision_21: 0.6376 - recall_21: 0.6219
Epoch 2/50
257/257 [==============================] - 58s 224ms/step - loss: 0.6119 - acc: 0.6557 - precision_21: 0.6712 - recall_21: 0.6243
Epoch 3/50
257/257 [==============================] - 54s 212ms/step - loss: 0.6016 - acc: 0.6644 - precision_21: 0.6836 - recall_21: 0.6253
Epoch 4/50
257/257 [==============================] - 50s 194ms/step - loss: 0.5874 - acc: 0.6776 - precision_21: 0.6965 - recall_21: 0.6414s - loss: 0.5878 - acc: 0.6772 - precision_21: 0.6954 - recall_21: 
Epoch 5/50
257/257 [==============================] - 48s 185ms/step - loss: 0.5856 - acc: 0.6832 - precision_21: 0.7029 - recall_21: 0.6460
Epoch 6/50
257/257 [==============================] - 54s 210ms/step - loss: 0.5825 - acc: 0.6848 - precision_21: 0.7020 - recall_21: 0.6535
Epoch 7/50
257/257 [==============================] - 50s 196ms/step - loss: 0.5757 - acc: 0.6831 

257/257 [==============================] - 50s 196ms/step - loss: 0.6087 - acc: 0.6602 - precision_22: 0.6781 - recall_22: 0.6109
Epoch 3/50
257/257 [==============================] - 53s 204ms/step - loss: 0.5976 - acc: 0.6695 - precision_22: 0.6851 - recall_22: 0.6285s - loss: 0.5977 - acc: 0.6689 - precision_22: 0.6844 - recal
Epoch 4/50
257/257 [==============================] - 54s 210ms/step - loss: 0.5924 - acc: 0.6727 - precision_22: 0.6929 - recall_22: 0.6214
Epoch 5/50
257/257 [==============================] - 56s 219ms/step - loss: 0.5880 - acc: 0.6750 - precision_22: 0.6908 - recall_22: 0.6348
Epoch 6/50
257/257 [==============================] - 55s 214ms/step - loss: 0.5837 - acc: 0.6820 - precision_22: 0.7059 - recall_22: 0.6248
Epoch 7/50
257/257 [==============================] - 53s 208ms/step - loss: 0.5816 - acc: 0.6868 - precision_22: 0.7062 - recall_22: 0.6409s - loss: 0.5814 - acc: 0.6870 - precision_ - ETA: 2s - loss: 0.5812 - acc: 0.6867 - precision_22: 0.
Epo

Epoch 1/50
257/257 [==============================] - 41s 159ms/step - loss: 0.6348 - acc: 0.6355 - precision_23: 0.6508 - recall_23: 0.5793
Epoch 2/50
257/257 [==============================] - 41s 158ms/step - loss: 0.6057 - acc: 0.6613 - precision_23: 0.6789 - recall_23: 0.6079
Epoch 3/50
257/257 [==============================] - 41s 159ms/step - loss: 0.6032 - acc: 0.6613 - precision_23: 0.6732 - recall_23: 0.6225
Epoch 4/50
257/257 [==============================] - 40s 156ms/step - loss: 0.5908 - acc: 0.6792 - precision_23: 0.6983 - recall_23: 0.6272
Epoch 5/50
257/257 [==============================] - 42s 162ms/step - loss: 0.5855 - acc: 0.6798 - precision_23: 0.6938 - recall_23: 0.6399s - loss: 0.587
Epoch 6/50
257/257 [==============================] - 40s 154ms/step - loss: 0.5782 - acc: 0.6838 - precision_23: 0.7057 - recall_23: 0.6269
Epoch 7/50
257/257 [==============================] - 41s 158ms/step - loss: 0.5750 - acc: 0.6867 - precision_23: 0.7033 - recall_23: 0.642

In [ ]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

In [ ]:
#cnn+lstm

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
pat = 5
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= glove (X_train,y_train,X_test,y_test)
    
    modelCNNLSTM = Sequential() #cnn

    modelCNNLSTM.add(embedding_layer)
    modelCNNLSTM.add(Conv1D(filters=256, kernel_size=5, activation='relu'))
    modelCNNLSTM.add(MaxPooling1D(pool_size=4))
    modelCNNLSTM.add(Dropout(0.25))
    modelCNNLSTM.add(LSTM(128))                            
    modelCNNLSTM.add(Flatten())
    modelCNNLSTM.add(Dense(1, activation='sigmoid'))
    modelCNNLSTM.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNNLSTM.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelCNNLSTM.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))